In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 10000

In [4]:
(X_train, y_train),(X_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)

/Users/kebhana/opt/anaconda3/envs/py-nlp/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/kebhana/opt/anaconda3/envs/py-nlp/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(25000,)
(25000,)
(25000,)
(25000,)


In [9]:
X_train[:1]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])],
      dtype=object)

In [10]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [12]:
print(X_train.shape)
print(X_test.shape)

(25000, 200)
(25000, 200)


In [14]:
y_train[:5]

array([1, 0, 0, 1, 0])

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPool1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [17]:
embedding_dim = 256
batch_size= 256

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 256))
model.add(Dropout(0.3))
model.add(Conv1D(256, 3, padding='valid', activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [19]:
es = EarlyStopping(monitor='val_loss', mode = 'min', verbose=1, patience=4)
mc = ModelCheckpoint('model/1d_cnn_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 0.4133 - acc: 0.7966
Epoch 00001: val_acc improved from -inf to 0.87292, saving model to model/1d_cnn_model.h5
782/782 [==============================] - 57s 73ms/step - loss: 0.4133 - acc: 0.7966 - val_loss: 0.2958 - val_acc: 0.8729
Epoch 2/20
781/782 [============================>.] - ETA: 0s - loss: 0.2200 - acc: 0.9135
Epoch 00002: val_acc improved from 0.87292 to 0.89048, saving model to model/1d_cnn_model.h5
782/782 [==============================] - 56s 72ms/step - loss: 0.2199 - acc: 0.9135 - val_loss: 0.2592 - val_acc: 0.8905
Epoch 3/20
781/782 [============================>.] - ETA: 0s - loss: 0.1135 - acc: 0.9600
Epoch 00003: val_acc did not improve from 0.89048
782/782 [==============================] - 57s 73ms/step - loss: 0.1135 - acc: 0.9600 - val_loss: 0.3528 - val_acc: 0.8663
Epoch 4/20
781/782 [============================>.] - ETA: 0s - loss: 0.0494 - acc: 0.9836
Epoch 00004: val_acc did not impro

In [21]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

OSError: SavedModel file does not exist at: best_model.h5/{saved_model.pbtxt|saved_model.pb}